In [1]:
import insightface
from insightface.app import FaceAnalysis
import cv2
import numpy as np

c:\Users\risky\anaconda3\envs\arcface\lib\site-packages\albumentations\__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.8 (you have 1.4.21). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
app = FaceAnalysis(name="buffalo_s")
app.prepare(ctx_id=0, det_size=(640, 640))

c:\Users\risky\anaconda3\envs\arcface\lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\risky/.insightface\models\buffalo_s\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\risky/.insightface\models\buffalo_s\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\risky/.insightface\models\buffalo_s\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\risky/.insightface\models\buffalo_s\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\risky/.insightface\models\buffalo_s\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127

In [3]:
img = cv2.imread("dataset_static/billie/dummy1.jpg")
faces = app.get(img)
emb = faces[0].normed_embedding

c:\Users\risky\anaconda3\envs\arcface\lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


In [4]:
from pathlib import Path
from sklearn.svm import SVC
import pickle

X = []
y = []

root = Path("dataset_static")
for person_dir in root.iterdir():
    if not person_dir.is_dir():
        continue
    label = person_dir.name
    for img_path in person_dir.glob("*.jpg"):
        img = cv2.imread(str(img_path))
        faces = app.get(img)
        if len(faces) == 0:
            continue
        emb = faces[0].normed_embedding
        X.append(emb)
        y.append(label)

X = np.array(X)
y = np.array(y)
    
clf = SVC(kernel="linear", probability=True)
clf.fit(X, y)

# simpan biar bisa dipakai FastAPI
with open("svm_face.pkl", "wb") as f:
    pickle.dump(clf, f)

c:\Users\risky\anaconda3\envs\arcface\lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


In [5]:
import os
from pathlib import Path

import insightface
import cv2
import numpy as np
import pickle

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# load insightface (pakai model yg sama dgn training)
app = insightface.app.FaceAnalysis(name="buffalo_s")
app.prepare(ctx_id=0, det_size=(640, 640))  # ctx_id=-1 kalau mau CPU

# load SVM hasil training
with open("svm_face.pkl", "rb") as f:
    svm_model = pickle.load(f)

print("Model & SVM loaded.")

c:\Users\risky\anaconda3\envs\arcface\lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\risky/.insightface\models\buffalo_s\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\risky/.insightface\models\buffalo_s\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\risky/.insightface\models\buffalo_s\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\risky/.insightface\models\buffalo_s\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\risky/.insightface\models\buffalo_s\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127

In [6]:
import os
from pathlib import Path

import insightface
import cv2
import numpy as np
import pickle

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# load insightface pakai model yang sama seperti waktu training
app = insightface.app.FaceAnalysis(name="buffalo_s")
app.prepare(ctx_id=0, det_size=(640, 640))  # ctx_id=-1 kalau CPU

# load svm
with open("svm_face.pkl", "rb") as f:
    svm_model = pickle.load(f)

print("Model & svm loaded")

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\risky/.insightface\models\buffalo_s\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\risky/.insightface\models\buffalo_s\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\risky/.insightface\models\buffalo_s\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\risky/.insightface\models\buffalo_s\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\risky/.insightface\models\buffalo_s\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127

In [7]:
test_root = Path("dataset_static")

if not test_root.exists():
    raise FileNotFoundError(f"Folder {test_root} tidak ditemukan")
else:
    print("✅ pakai folder:", test_root)

✅ pakai folder: dataset_static


In [8]:
def get_embedding(path: Path):
    img = cv2.imread(str(path))
    if img is None:
        return None
    faces = app.get(img)
    if len(faces) == 0:
        return None
    return faces[0].normed_embedding

In [9]:
y_true = []
y_pred = []
skipped = []   # buat nyatet gambar yang nggak kedeteksi

for person_dir in test_root.iterdir():
    if not person_dir.is_dir():
        continue

    label = person_dir.name  # nama orang = nama folder

    for img_path in person_dir.glob("*.jpg"):
        emb = get_embedding(img_path)
        if emb is None:
            skipped.append(str(img_path))
            continue

        # prediksi pakai svm
        pred_label = svm_model.predict([emb])[0]

        y_true.append(label)
        y_pred.append(pred_label)

print(f"✅ total sample yang dipakai: {len(y_true)}")
print(f"⚠️ total yang diskip (no face / gagal baca): {len(skipped)}")

c:\Users\risky\anaconda3\envs\arcface\lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


✅ total sample yang dipakai: 2
⚠️ total yang diskip (no face / gagal baca): 0


c:\Users\risky\anaconda3\envs\arcface\lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


In [10]:
if len(y_true) == 0:
    print("❌ tidak ada sample yang berhasil diproses. cek lagi folder gambar atau deteksinya.")
else:
    acc = accuracy_score(y_true, y_pred)
    print(f"✅ Accuracy: {acc:.4f}")

✅ Accuracy: 1.0000


In [11]:
if len(y_true) > 0:
    print("=== classification report ===")
    print(classification_report(y_true, y_pred))

    labels = sorted(list(set(y_true)))
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    print("=== confusion matrix (baris = actual, kolom = pred) ===")
    print("labels:", labels)
    print(cm)

=== classification report ===
              precision    recall  f1-score   support

      billie       1.00      1.00      1.00         1
     madison       1.00      1.00      1.00         1

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2

=== confusion matrix (baris = actual, kolom = pred) ===
labels: ['billie', 'madison']
[[1 0]
 [0 1]]
